In [0]:
dbutils.widgets.text("environment", "")

In [0]:
dbutils.widgets.get("p_environment")
v_environment = dbutils.widgets.get("p_environment")
print(v_environment)

## Ingestion del archivo movie_cast.json

### Paso 1- Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
# {
#     "roleId": 1,
#     "languageRole": "Original"
#   }

language_rol_schema = StructType(fields=[
    StructField("roleId", IntegerType(), True),
    StructField("languageRole", StringType(), True)
 ])


In [0]:
language_rol_df = spark.read \
    .schema(language_rol_schema) \
    .option("multiLine", True) \
    .json("/mnt/moviehistoryl/bronce/language_role.json")

In [0]:
language_rol_df.printSchema()

In [0]:
language_rol_df.display()

### Paso 2 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "evironment"
1. "roleId" renombrar a "role_id"
2. "languageRole" renombrar a "language_role"
3. Agregar las columnas "ingestion_date" y environment"


In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
language_rol_final_df = language_rol_df \
    .withColumnRenamed("roleId", "role_id") \
    .withColumnRenamed("languageRole", "language_role") \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("environment", lit(v_environment)) \
    
language_rol_final_df.display()

### Paso 3 - Escribir la salida en formato parquet

In [0]:
language_rol_final_df.write.mode("overwrite").parquet("/mnt/moviehistoryl/silver/language_rol")

In [0]:
spark.read.parquet("/mnt/moviehistoryl/silver/language_rol").display()

In [0]:
dbutils.notebook.exit("succes")